# A1 - Assignment - Part 1

### Team A2: Lennart, Henrique, Anya, Alejandra and Kaustubh

<div class="alert alert-success">
1. Provide a list of 30 Marvel characters
</div>




<div class="alert alert-warning">
To successfully run the following code please install the following libraries:
</div>

###   requests
    pip install requests
###   hashlib
    pip install hashlib
###   datetime
    pip install datetime
###   pandas
    pip install pandas

In [1]:
# Import relevant libraries
import requests
import hashlib
import datetime
import pandas as pd

# Keys and URL gateway for Marvel API
priv_key = "1e42b2e9b79510fb83ac6140d15fb2d429774942"

pub_key = "ee497591e5a579893fbb0dfc4c42e44d"

urlMarv = 'https://gateway.marvel.com:443/v1/public/characters'

timestamp = datetime.datetime.now().strftime('%Y-%m-%d%H:%M:%S') # generates timestamp in the format YYYY-mm-dd HH:MM:SS


# Marvel API requires server side API calls to include md5 hash of timestamp + public key + private key 

def hash_params():
    
    hash_md5 = hashlib.md5()
    hash_md5.update(f'{timestamp}{priv_key}{pub_key}'.encode('utf-8'))
    hashed_params = hash_md5.hexdigest()

    return hashed_params

In [2]:
# Setting up the params and GET request to the URL
params = {'ts': timestamp, 'apikey': pub_key, 'hash': hash_params(), 'limit': 30, 'offset': 30 };

res = requests.get(urlMarv,params=params)

results = res.json()

<div class="alert alert-success">
2. Retrieve the Ids for all the characters in your list (in str form)
3. Retrieve the total number of Events available for all the characters in your list (in integer form)
4. Retrieve the total number of Series available for all the characters in your list  (in integer form)
5. Retrieve the total number of Comics available for all the characters in your list (in integer form)</div>

In [3]:
# Function to retrieve value in dataframe with 1 variable
def retrieve_val1(val_prov, results):
    char = []
    for i in results['data']['results']:
        val = (i[val_prov])
        char.append(val)
    return char

In [4]:
# Function to retrieve value in dataframe with 2 variables
def retrieve_val2(val_prov1, val_prov2, results):
    char = []
    for i in results['data']['results']:
        val = (i[val_prov1][val_prov2])
        char.append(val)
    return char

<div class="alert alert-success">
6. Retrieve the Price of the most expensive comic that the character was featured in for all the characters in your list (in float form and USD)</div>

In [5]:
# Nested loop to retrieve and sort the price of most expensive comic for a character

comic_results_list = []
highest_price = []

for element in retrieve_val1('id', results):
    characterId = element
    base_url4 = f'https://gateway.marvel.com:443/v1/public/characters/{characterId}/comics?'
    query_url4 = base_url4 + 'ts='+ timestamp + '&apikey=' + pub_key+ '&hash='+ hash_params()
    response_comics4 = requests.get(query_url4)
    comic_results = response_comics4.json()
    comic_results_list.append(comic_results)

    
all_prices_comics = []

for dicts in comic_results_list:
    path = dicts ['data']['results']
    for dicts2 in path:
        path2 = dicts2 ['prices']
        for dicts3 in path2:
            all_prices_comics.append(dicts3['price'])
    highest_price.append(max(all_prices_comics,default=0))
    all_prices_comics = []

<div class="alert alert-success">
7.1 Store the data above in a pandas DataFrame called df containing exactly in the following columns: Character Name, Character ID, Total Available Events, Total Available Series, Total Available Comics, Price of the Most Expensive Comic.
</div>

In [6]:
# Creating a dataframe with the column names as per instruction
df = pd.DataFrame(columns = ["Character Name", "Character ID", "Total Available Events", "Total Available Series", "Total Available Comics"])

# Adding value to the dataframe per column by through functions of retrieve_val1 & retrieve_val2
df["Character Name"] = retrieve_val1('name', results)
df["Character ID"] = retrieve_val1('id', results)
df["Character ID"] = df["Character ID"].astype(str)
df["Total Available Events"] = retrieve_val2('events','available', results)
df["Total Available Events"] = df["Total Available Events"].astype(int)
df["Total Available Series"] = retrieve_val2('series','available', results)
df["Total Available Series"] = df["Total Available Series"].astype(int)
df["Total Available Comics"] = retrieve_val2('comics','available', results)
df["Total Available Comics"] = df["Total Available Comics"].astype(int)
df["Price of the Most Expensive Comic"] = highest_price
df["Price of the Most Expensive Comic"] = df["Price of the Most Expensive Comic"].astype(float)

<div class="alert alert-success">
7.2 If a character is not featured in Events, Series or Comics the corresponding entry should be filled in with a None (of NoneType). If a character does not have a Price the corresponding entry should be filled in with a None (of NoneType).
</div>

In [7]:
for i in range(len(df)): # For loop to go through the dataframe
    if df.loc[i, "Total Available Events"] == 0: #If statement to check if any value is 0
        df.loc[i, "Total Available Events"] = None #If True then replace 0 with None
        
    if df.loc[i, "Total Available Series"] == 0: #If statement to check if any value is 0
        df.loc[i, "Total Available Series"] = None #If True then replace 0 with None
        
    if df.loc[i, "Total Available Comics"] == 0: #If statement to check if any value is 0
        df.loc[i, "Total Available Comics"] = None #If True then replace 0 with None
        
    if df.loc[i, "Price of the Most Expensive Comic"] == 0: #If statement to check if any value is 0
        df.loc[i, "Price of the Most Expensive Comic"] = None #If True then replace 0 with None

 <div class="alert alert-success">
8. Save df to a file called data.csv</div>

In [8]:
# Saving dataframe to CSV file
df.to_csv("data.csv", index=False)

 <div class="alert alert-success">
Some examples of the requests created in Part 2</div>

In [9]:
response1 = requests.get('http://127.0.0.1:5000/characters', params = {'Character_ID':'1009151'}).json()
print (response1)

{'status': 200, 'response': [{'Character Name': 'Amiko', 'Character ID': 1009151, 'Total Available Events': 1.0, 'Total Available Series': 3.0, 'Total Available Comics': 12.0, 'Price of the Most Expensive Comic': 1.99}]}


In [10]:
response2 = requests.post('http://127.0.0.1:5000/characters', params = {'Character_ID': '10987',
                                                                     'Character_Name': 'Superman',
                                                                     'Total_Available_Events': 12,
                                                                     'Total_Available_Series': 2,
                                                                     'Total_Available_Comics': 3,
                                                                     'Price_of_the_most_expensive_comic': 34}).json()
print(response2)

{'status': 200, 'response': [{'Character Name': 'Superman', 'Character ID': 10987, 'Total Available Events': 12.0, 'Total Available Series': 2.0, 'Total Available Comics': 3.0, 'Price of the Most Expensive Comic': 34.0}]}


In [11]:
response3 = requests.delete('http://127.0.0.1:5000/characters', params = {'Character_ID':'1009151'}).json()
print (response3)

{'message': "Did not attempt to load JSON data because the request Content-Type was not 'application/json'."}
